# Recurrent Neural Network:

The Simple Neural Network takes features as input $x$ and optimize error. While Recurrent Neural Network not only learns from features but also take cares of sequence values over the time. It stores the information at each state and passes it to other state of network so mimic the 'memory'.

In this notebook we will do following:
1. Description of Recurrent Recurrnet Network
2. Deployment of Recurrent Network
3. Performance of Network on different length of training examples

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # ploting graph

### Generate Data
To discuss Generalized behavior of Regression model, by Generalized we mean it can work on any shape of data, we have created the generate_bits functions. The function is simple as it takes desired number of rows m and number of feature n_x and it randomly generated binary data i.e. 0 and 1.

In [ ]:
def generate_bits(n_x, m):
# Generate a m x n_x array of ints between 0 and 1, inclusive:
# m: number of rows
# n_x : number of columns per rows/ feature set
    np.random.seed(1)
    data = np.random.randint(2, size=(n_x, m))
    return(data)

Create Labels:
For training/updating derivatives of parameters weight and bias, the loss function determine the difference between actual values and the activation values. The actual value is the value that each example(row) has as it label. Like the the actual value of OR operation:
$$1+0=1=actualValue>:[oroperation=+]$$
The generate_label function below takes data as input and apply XOR operation row wise.

In [ ]:
def generate_label(data, m):
    # generate label by appyling xor operation to individual row
    # return list of label (results)
        # data: binary data set of m by n_x size
    lst_y = []
    y= np.empty((m,1))
    k = 0
    for tmp in data.T:
        xor = np.logical_xor(tmp[0], tmp[1])

        for i in range(2, tmp.shape[0]):
            xor = np.logical_xor(xor, tmp[i])
    #     print(xor)
        lst_y.append(int(xor))
        y[k,:] = int(xor)
        k+=1
    return(y.T)

## Forward Propagation:

The Recurrenet Neural Network, is network that loop over it, passing information to itself timely. We may say that at input to the network we have input features and activations on the input is computed in the RNN-Cell whose activation is used for computing next step activation based on next input features and for the prediction purpose. This iteration goes up the length of the input sequence.  Let's see the following figure to get insipration of architecture:


![](https://github.com/hamzafar/deep_learning_toys/blob/master/images/rnn/3.JPG?raw=true)

The above is generic architecture, which takes input feature $x$ at time step $t$ and whose activation $a$ and output $y^{t}$ is computed on the $x$ and previous time step activation$a^{t-1}$. Unrolling the above network resulted in following architecture. Where the each input and output values to RNN-cell is marked with time step $0$ to $t$.


![](https://github.com/hamzafar/deep_learning_toys/blob/master/images/rnn/4.JPG?raw=true)

The operations in RNN-Cell consists of computing linear activations of input feature $x$ by multiplying it with some paramters $W_{ax}$ and adding some bias $b_a$. The previous activation information is also added to this step as $a^{t-1}$ is mulyiplied by parameter $W_{aa}$ and this is also added to the results of values we have achieved by input value $x$, parameter $W_{ax}$ and bias $b_a$.
The computed value is then passed from some non-linear function, we use $tanh$ that resulted in activations $a$. The same activation is used to compute prediction by using some function $sigmoid$ or $softmax$.
The following diagram shows the process for sinle RNN-cell:

![](https://image.ibb.co/fdXpbx/Capture.jpg)

So, the equations to compute the RNN-Cell will be as follow:
$$a^{(t)} = tanh(W_{ax}*x^{(t)} + W_{aa}*a^{(t-1)} + b_a)$$
$$\hat{y}^{(t)} = sigmoid(W_{ya}*a^{(t)} + b_y)$$

**sigmoid:** We are using sigmoid function to predcit $\hat{y}$ as we only have two posible outcomes as $0$ or $1$ as target variable.

In [ ]:
def sigmoid(z):
    # determine sigmoid of value
        #z: linear activation
    s = 1 / (1 + np.exp(-z))
    return s

**rnn_cell_compute:** This helper function is responsible to determine activations and predicted values for single RNN-Cell. The Function takes input feature $x$ at single time stemp, parameters and previous activations.

In [ ]:
def rnn_cell_compute(xt, a_prev, parameters):
    # Computes and returns activations, predcitions yhat
        #xt: input to current state
        #a_prev: activation from previous state
        #parameters: dictonary object of weights and bias
        
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    # compute next activation state using the formula given above
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    # compute output of the current cell using the formula given above
    yt_pred = sigmoid(np.dot(Wya, a_next) + by)

    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

So, the RNN unfolded architecture will be combining each single RNN-Cell and get input of features $x$ and activations from the previovs sates $a^{(t-1)}$. We have combined each RNN-Cell to demonstrate the architecture over $t$ time sequence.
![](https://image.ibb.co/bWAjbx/Untitled.png)

**rnn_forward:** the generic function, that computes the activations and predictions based on the length of sequence of data. This function iterates about the length of sequence $t$ and get activations and predictions from the **rnn_cell_compute**. The function takes $x$ input data, previous activations and parameters.

In [ ]:
def rnn_forward(x, a0, parameters):
    # Itereate over the rnn_cell_compute function to compute activations, predictions for the states length equal to time stamp
        #x: input data with feature set and time stemp t_x
        #a0: activations at previous state
        #parameters: dictonary object of weights and bias

    caches = []
    
    # Retrieve dimensions from shapes of x and Wy
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y" with zeros 
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next 
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache 
        a_next, yt_pred, cache = rnn_cell_compute(x[:,:,t], a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y 
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches" 
        caches.append(cache)
            
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

**initialize_param:**  This function generates random arrays for the paramaters used in Recuurent Netowrk.  It takes arguments like number of activation in network $h$, input data feature $n_x$ and number of output node $n_y$ and returnd dictionary of parameters.

In [ ]:
def initialize_param(h, n_x, n_y):
    # Intialize required paramters w's and b's
        #h: number of activations
        #n_x: number of input feature set
        #n_y: number of output node
    np.random.seed(1)
    Waa = np.random.randn(h,h)
    Wax = np.random.randn(h,n_x)
    Wya = np.random.randn(n_y,h)
    ba = np.random.randn(h,n_y)
    by = np.random.randn(1,1)
    parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}
    return(parameters)

**compute_cost:** The compute cost function, calculate loss of each of the training example in the data set by comparing difference between actual and produce activation from the network and then average all the loss to get a single cost value.

In [ ]:
def compute_cost(y, y_pred, t_x, m):
    #calculate logistic cost w.r.t. prediction yhat and actual value y
        #y: target label
        #y_pred: predicted values from network
        #t_x: length of time stemp
        #m: number of trainig examples
    y_hat = y_pred[:,:,t_x-1]
    loss = -1*(y* np.log(y_hat) + (1-y) * np.log(1-y_hat))
    cost = np.sum(loss)/m
    return(cost)

## Backward Propagation:

![](https://github.com/hamzafar/deep_learning_toys/blob/master/images/rnn/5.jpg?raw=true)

Refer to above figure, it can be observed that in the back-propagation, we divide process into two catigories:
1. Deriavtive of output layer 
2. Derviative of RNN-Cell

In, **Deriavtive of output layer**, we will compute derivatives of parameters related to output layer, in our case $b_y$ and $W_{ya}$ are the parameters, we are considering. The equations to take derivative of parameters w.r.t loss are as follows:

$$\frac{\partial{loss}}{\partial{z_y}} = \hat{y}-y$$
$$d_{z_y} = \frac{\partial{loss}}{\partial{z_y}}$$
$$\frac{\partial{loss}}{\partial{b_y}} = d_{z_y}$$
$$\frac{\partial{loss}}{\partial{W_{ya}}} = d_{z_y} * a^{<tx>}$$

**derivative_output: ** The derived equations above are computed in this helper funciton. The primarly task of this helper functions is to get derivatives of parameters that are connected to output layer i.e. $W_{ya}$ and $b_y$

In [ ]:
def derivative_output(yhat, y, a, t_x, m):
     ## compute partial derivative of parameters at output layer
        # yhat: activation from sigmoid 
        # y: target label
        # a: activation at last time stamp
    dZy = yhat - y
    dby = (1 / m) * np.sum(dZy, axis=1, keepdims=True)
    dWya = (1 / m) * np.dot(dZy, a[:,:,t_x-1].T) 
    
    gradient_output = {'dWya': dWya, 'dby': dby, 'dZy': dZy}
    return(gradient_output)

The RNN-Cell contains three parameters $W_{aa}$, $W_{ax}$ and $b_a$, we want to update in **Derviative of RNN-Cell** process. The following are the equations for taking derivatives of parameters:


$$d_{tanh} = (1-a^{<tx>^2})*da^{<tx>} $$
$$d_z = d_{tanh}$$
$$d_{ba} = d_{tanh}$$
$$d_{Waa} = d_{tanh} * a^{<tx-1>} $$
$$d_{Wax} = d_{tanh} * x^{<tx>}$$

The $da^{<tx>}$ will be derived differently. This is generally, equated in last time stamp state and the rest of the early states.
1. For last time stamp: $da^{<tx>} = W_{ya} * d_{zy}$
2. And for early states:  $da^{<tx>} = W_{aa} * d_{z} ^{<tx+1>}$

**rnn_cell_backward:** The paritial derivative of loss w.r.t  parmaeters in the rnn cell are computed in this function. In this case the parameters are $W_{aa}$, $W_{ax}$ and $b_{a}$. 

In [ ]:
def rnn_cell_backward(da_next, cache):
    # compute derivatives and returns a dictonary objects of them
        #da_next: derivative of activation of next state
        #cache: tuple containing information from the forward pass (output of rnn_cell_forward())
    
    # Retrieve values from cache
    (a_next, a_prev, xt, parameters) = cache
    
    # Retrieve values from parameters
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]

    # compute the gradient of tanh with respect to a_next
    dtanh = (1 - a_next ** 2) * da_next

    # compute the gradient of the loss with respect to Wax 
    dxt = (1 / m) * Wax.T.dot(dtanh)
    dWax = (1 / m) * dtanh.dot(xt.T)

    # compute the gradient with respect to Waa
    da_prev = (1 / m) * Waa.T.dot(dtanh)
    dWaa = (1 / m) * dtanh.dot(a_prev.T)

    # compute the gradient with respect to b 
    dba = (1 / m) * np.sum(dtanh, 1, keepdims=True)

    
    
    # Store the gradients in a python dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}
    
    return gradients

**rnn_backward:**  This helper function iterates on *rnn_cell_backward*. The flow of loop is backward from output layer to cell state at time one.  During the iteration this function also calculates $da^{<tx>}$ at output layer or early states other than output.

In [ ]:
def rnn_backward(caches, y, n_a, m, T_x, yhat, a): 
    # compute every states derivatives and return them
        #cache: tuple containing information from the forward pass (rnn_forward)
        #y: target label
        #n_a: number of activation states
        #m: training examples
        #T_x: time stam
        #yhat: predicted values
        #a: activations
        
    # Retrieve values from the first cache (t=1) of caches 
    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[0]
    
    # Retrieve dimensions from x1's shapes 
    n_x, m = x1.shape
    
    da = np.empty((h,m))
    
    # initialize the gradients with the right sizes (≈6 lines)
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da0 = np.zeros((n_a, 1))
    da_prevt = np.zeros((n_a, 1))
    
    # Loop through all the time steps
    for t in reversed(range(T_x)):
        # Compute gradients at time step t. Choose wisely the "da_next" and the "cache" to use in the backward propagation step.
        if (t == T_x-1):
            gradient_output = derivative_output(yhat[:,:, t], y, a, T_x, m)

            dZy = gradient_output['dZy']
            Wya = parameters['Wya']
            da = np.dot(Wya.T,dZy)
        else:
            Waa = parameters['Waa']
            da = np.dot(Waa.T,da_prevt)      
        
        gradients = rnn_cell_backward(da, caches[t])
        
        # Retrieve derivatives from gradients (≈ 1 line)
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dWaa"], gradients["dba"]
        # Increment global derivatives w.r.t parameters by adding their derivative at time-step t (≈4 lines)
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
        
    # Set da0 to the gradient of a which has been backpropagated through all time-steps (≈1 line) 
    da0 = da_prevt

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba, "dWya": gradient_output["dWya"],
                 "dby": gradient_output["dby"] }
    
    return gradients

**update_parameters:** This function update paramaters w's and b's according to the following equation:
$$w := w - \partial{w} * learningRate $$
$$b := b - \partial{b} * learningRate $$

In [ ]:
def update_parameters(parameters, gradients, lr):
    # update and return parameters 
        # parameters: dictonary object of w's and b's
        # gradients: dictionary: object of gradient of w's and b's
        #lr: learning rate
        
    parameters['Waa'] = parameters['Waa'] - (lr * gradients['dWaa'])
    parameters['Wax'] = parameters['Wax'] - (lr * gradients['dWax'])
    parameters['Wya'] = parameters['Wya'] - (lr * gradients['dWya'])
    parameters['ba'] = parameters['ba'] - (lr * gradients['dba'])
    parameters['by'] = parameters['by'] - (lr * gradients['dby'])
    
    return(parameters)

**update_prev_a:** The activations initalized randomnly before first time stamp is also updated as:
$$a_0 = a_0 -\partial{a_0} * learningRate$$

In [ ]:
def update_prev_a(a0, gradients, lr):
    # update and return activations
        #a0: activations before first state
        #gradients: dictionary: object of gradient of w's and b's
        #lr: learning rate
        
    a0 = a0 - (lr*gradients['da0'])
    return(a0)

**optimize_parameters: ** this function call all the function i.e. the functions used in forward propagation, used in getting cost and used in back propagation to determine derivatives of paratmeters. and then update the parameters.
The above process run $n$ times specified by the user. 

In [ ]:
def optimize_parameters(x, y, a0, parameters, h, m, t_x, lr, num_iter):
    # returns list of cost at each iteration and updated parameters
        #x: input data
        #y: target labels
        #a0: activation before state 1
        #parameters: dictionary objects of parameters
        #h: number of node
        #m: number of training examples
        #t_x: time stamp (length of sequence of data)
        #lr: learning rate
        #num_iter: an integer
    lst_cost = []

    for i in range(0, num_iter):
        a, yhat, caches = rnn_forward(x, a0, parameters)
        gradients = rnn_backward(caches, y, h, m, t_x, yhat, a)
        parameters = update_parameters(parameters, gradients, lr)
        cost = compute_cost(y, yhat, t_x, m)
        lst_cost.append(cost)
        a0 = update_prev_a(a0, gradients, lr)
    return (lst_cost, parameters)

In [ ]:
def plt_res(lst, ylab, lr):
    #This will plot the list of values at y axis while x axis will contain number of iteration
    #lst: lst of action/cost
    #ylab: y-axis label
    #lr: learning rate
    plt.plot(lst)
    plt.ylabel(ylab)
    plt.xlabel('iterations')
    plt.title("Learning rate =" + str(lr))
    plt.show()

We have created two different datasets containing $10000$ and $100000$ training examples with sequence lenth $50$. The fixed RNN architecture is trained on both of the datasets and their cost over time is determined.  The network contaion $25$ hidden states $h$ with learning rate $0.07$. It is trained about $1000$ iterations.

In [ ]:
np.random.seed(1)

n_x = 1
t_x = 50
n_y = 1
m = 10000
lr = 0.07
h = 25 
num_iter = 1000

x = generate_bits(m, t_x)
y = generate_label(x.T, m)
x = x.T.reshape(n_x,m,t_x)

a0 = np.random.randn(h,m)
parameters = initialize_param(h, n_x, n_y)

"""Uncomment below line of code to trian RNN"""
# lst_cost_s, parameters_s = optimize_parameters(x, y, a0, parameters, h, m, t_x, lr, num_iter)

######################################################################################

m = 100000

x = generate_bits(m, t_x)
y = generate_label(x.T, m)
x = x.T.reshape(n_x,m,t_x)

a0 = np.random.randn(h,m)
parameters = initialize_param(h, n_x, n_y)

"""Uncomment below line of code to trian RNN"""
# lst_cost_m, parameters_m = optimize_parameters(x, y, a0, parameters, h, m, t_x, lr, num_iter)

We have trained rnn on $10,000$ and $100,000$ training samples on local machine and the cost graphs are shown below,The x-axis shows the cost of training while y-axis plot the number of iteration:


Training Examples|Cost Graph|
------------- |:-------------:|
**10,000** | ![](https://github.com/hamzafar/deep_learning_toys/blob/master/images/rnn/rnn-10000-1000-007.png?raw=true)|
**100,000** | ![](https://github.com/hamzafar/deep_learning_toys/blob/master/images/rnn/rnn-100000-1000-007.png?raw=true)|




---